**Load the required libraries and data**

In [1]:
# Load the required libraries 
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
import seaborn as sns 
import ast 

In [2]:
# Load the data files 

# Information about the promotional offers that are possible to receive, and basic information 
# about each one including the promotional type, duration of the promotion, reward, and how the promotion 
# was distributed to customers.
portfolio = pd.read_csv("data/portfolio.csv")

# Dimensional data about each person, including their age, salary, and gender, there is one unique customer 
# for each record.
profile = pd.read_csv("data/profile.csv")

# Records show the different steps of promotional offers that a customer received. The different values of 
# receiving a promotion are receiving, viewing, and completing. You also see the different transactions 
# that a person made in the time since he became a customer. With all records, you see the day that they 
# interacted with Starbucks and the amount that is worth
transcript = pd.read_csv("data/transcript.csv")

# Explanatory Data Analysis 
## Data Inspection and Wrangling 

View the first few records of each of the dataset 

In [3]:
portfolio.head()

,Unnamed: 0,reward,channels,difficulty,duration,offer_type,id
0,0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [4]:
profile.head()

,Unnamed: 0,gender,age,id,became_member_on,income
0,0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
transcript.head()

,Unnamed: 0,person,event,value,time
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


**Drop unnecessary columns**

In [6]:
# store the datasets to a list 
data  = [portfolio, profile, transcript]

for dataset in data:
    dataset.drop("Unnamed: 0", axis=1, inplace=True)
    
# confirm the operation 
profile.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


How are the dataset structured? Lets view its info 

In [7]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


The portfolio dataset is complete with no null values, There are 10 records and 6 features. The dataset contains both numeric and non numeric features

In [8]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


The profile dataset contains null values, infact there are more than 4000 null values. There are 17000 records and a total of 5 features. Most features are numeric  

In [9]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


Transcript dataset is the largest dataset so far. Its complete with no null values. The dataset contains more than 30000 records and has only 4 features. Most of the features are non numeric while only one feature is numeric. 

### Wrangling and Cleaning 
#### Portfolio dataset 

In [10]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"['web', 'email', 'mobile', 'social']",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"['web', 'email', 'mobile', 'social']",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"['email', 'mobile', 'social']",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"['web', 'email', 'mobile', 'social']",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"['web', 'email', 'mobile']",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [11]:
channels = [] #stores all the channels for each individual portfolio

#Merge the channels into a single list
for channel in portfolio["channels"]:
    channel_list = ast.literal_eval(channel)
    
    for ch in channel_list:
        channels.append(ch)
    
# Find the unique channels 
unique_channels = list(np.unique(np.array(channels)))

# Find if channels exists in each individual portfolio 
for channel in unique_channels:
    globals()[channel] = [1 if channel in ch else 0 for ch in portfolio["channels"]]
    # Create a new column for the channel
    portfolio[channel] = globals()[channel]
    
# Delete the "channels" column 
portfolio.drop("channels", axis=1, inplace=True)

# Confirm the operation 
portfolio

,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


#### Profile

In [12]:
profile.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [13]:
# Lets find any id in the portfolios listed in the profiles
portfolio_ids =  list(portfolio["id"])
profile_ids = list(profile["id"])

for id_ in portfolio_ids:
    if id_ in profile_ids:
        print(id_)

An empty output shows no id from the portfolio was listed in the profile 

Convert the "became_member_on" to datetime datatype

In [14]:
became_member_on = pd.to_datetime(profile["became_member_on"], errors="coerce")
profile["became_member_on"] = [str(date.date()) for date in became_member_on]
profile.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,1970-01-01,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,1970-01-01,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,1970-01-01,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,1970-01-01,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,1970-01-01,NaN


Lets count the total null values to find the source of null values  

In [15]:
profile.isna().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

All null values are from gender and income variables. Lets heck the count of null values corresponding in both gender and income variables

In [16]:
sum(profile['gender'].isnull() & profile['income'].isnull())

2175

Since income and gender are very important variables and both have corressponding null variables therefore there is a need to drop rows containing the null values 

In [17]:
profile.dropna(how = "any", inplace=True)
# confirm the operation 
profile.isna().sum().sum()

0

In [18]:
profile.shape

(14825, 5)

There are almost 15000 profiles left. Are there any duplicate records? Lets find out 

In [19]:
profile.duplicated().sum()

0

There are no duplicated records in the profile dataset. The profile dataset is now clean and ready for further analysis. Lets its final look

In [20]:
profile

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,1970-01-01,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,1970-01-01,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,1970-01-01,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,1970-01-01,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,1970-01-01,51000.0
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,1970-01-01,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,1970-01-01,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,1970-01-01,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,1970-01-01,50000.0


#### Transcript

In [21]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


Find the dataset shape

In [22]:
transcript.shape

(306534, 4)

The "value" column in transcript has dictionary/json values. Lets find what they contain(their key).

In [23]:
# Extract values into a dictionaries and store them in a list 
values = [ast.literal_eval(value) for value in transcript["value"]]

# Get the keys of the dictionaries and store them to a list 
# distinct_keys = set(key for value in values for key in value.key())

value_keys = list() # to store all available keys in the dictionaries 

for value in values:
    keys = list(value.keys())
    for key in keys:
        value_keys.append(key)
        
distinct_keys = set(value_keys)
distinct_keys

{'amount', 'offer id', 'offer_id', 'reward'}

There are two similar key elements in the value feature, "offer_id" and "offer id". The "offer id" element need to be changed to "offer_id"

In [24]:
# Change "offer id" to "offer_id"
transcript["value"].replace("offer id", "offer_id", inplace=True, regex=True)

# Confirm the changes 
values = [ast.literal_eval(value) for value in transcript["value"]]

distinct_keys = set(key for value in values for key in value.keys())
distinct_keys

{'amount', 'offer_id', 'reward'}

Wow! The changes took place in the dataset.

In [25]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


How complete is the dataset? Lets find the null values present and if there are any duplicated rows therein. 

In [26]:
transcript.isna().sum()

person    0
event     0
value     0
time      0
dtype: int64

There are no null values in the dataset

In [27]:
transcript.duplicated().sum()

397

There are duplicated records in the dataset that need to be removed. They will be removed while keeping the first one. This will affect the index but will be fixed by reindexing the dataset

In [28]:
# Drop the the duplicated records 
transcript.drop_duplicates(keep="first", inplace=True)

# Reseting index 
transcript.reset_index(drop=True, inplace=True)

# confirm the operation
print("Duplicated records: ", transcript.duplicated().sum())
# dataset dimensions 
print("Data shape: ", transcript.shape)
# check if last index corresponds to the dataset dimensions 
transcript.tail()

Duplicated records:  0
Data shape:  (306137, 4)


,person,event,value,time
306132,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306133,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306134,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306135,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714
306136,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714


The last index corresponds with the size of the dataset. One last workaround is to check if the data types correspond with what is seen. This is especially for numeric and datetime features

In [30]:
transcript.dtypes

person    object
event     object
value     object
time       int64
dtype: object

The "time" feature is seen as numeric and also recognized by the machine as integer. **All the dataset are now clean**. Outliers will be removed upon visualization before making dashboards. 